Import libraries

In [ ]:
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf

To demonstrate the mechanics of classification, we will work with a dataset that is related to something we all know about, health. However, the methods, techniques, and caveats apply regardless of the classification task. We will be looking at a binary classification problem, which is a classification problems where the thing we are trying to learn to predict, i.e., the target, has two possible options. The following code block loads the data that we will be using. From https://www.kaggle.com/datasets/mathchi/diabetes-data-set:

```
This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective is to predict based on diagnostic measurements whether a patient has diabetes.

Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

 - Pregnancies: Number of times pregnant
 - Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
 - BloodPressure: Diastolic blood pressure (mm Hg)
 - SkinThickness: Triceps skin fold thickness (mm)
 - Insulin: 2-Hour serum insulin (mu U/ml)
 - BMI: Body mass index (weight in kg/(height in m)^2)
 - DiabetesPedigreeFunction: Diabetes pedigree function
 - Age: Age (years)
 - Outcome: Class variable. 0 represents no diabetes and 1 represents diabetes
```

In [ ]:
data = pd.read_csv('data/diabetes.csv')
data.head()

The following code block prints the shape of the data.

In [ ]:
data.shape

The following code block specifies the name of the column that contains the `target` variable that we want to predict. We then use the `.columns` attribute of the `DataFrame` to get a least of `features`. These features will be the things we believe are useful for predicting the value of the target.

In [ ]:
target = 'Outcome'
features = [col for col in data.columns if col != target]

The following code block plots the percent of instances that are associated with each possible value of the target variable. As you can see, the dataset we are working with has **class imbalance**. This simply means that the percent of instances that are associated with each class is not equal. In our case, the imbalance is significant, but not extreme. However, this is something we always need to consider because from a learning perspective, your model with have more information about one class than another and this can lead to some interesting problems.

In [ ]:
class_proportion = data.groupby(
    target
).agg(
    instances=(features[0], 'count')
).reset_index()

class_proportion['Percent'] = class_proportion['instances']/class_proportion['instances'].sum()

fig, ax = plt.subplots(1, 1, figsize=(6, 4))

class_proportion.plot(
    kind='bar',
    x='Outcome',
    y='Percent',
    edgecolor='k',
    ax=ax,
    legend=False,
)

ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Percent of Instances')
ax.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1.0))

plt.show()

Another thing that we want to check is the correlation between features. If we include two features that are highly correlated with one another, either in a positive or negative direction, the model can have trouble discerning which feature is contributing to the predictive task. Although we do have some correlations that are in the moderate range, there is nothing particularly troubling.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 6))

sns.heatmap(
    data.corr(),
    cmap='coolwarm',
    annot=True,
    fmt='.2f',
    linecolor='k',
    linewidths=0.1,
    cbar=False,
)

plt.show()

We will next split the data into two datasets:
1. a **training** set that we will use to fit the prediction model, and
2. a **testing** set that we will use to test the model.

The reason we split the data is to prevent the algorithm we use from simply *learning the data* and *overfitting*. Given the large computational resources and sophisticated models we have available today, this is a necessary step. Today we will be using a *logistic regression* model from the `statsmodels` library, so we need to use `pandas` to construct the datasets. In subsequent classes, we will be using `scikit-learn`, and they have some built in functions available to make this process easier for working with thier models. The following code block uses the `.sample` method to generate a training dataset that contains 75% of the rows of the original data.

In [ ]:
train = data.sample(frac=0.75, random_state=42)
train.shape

The following code block creates the testing dataset, which contains all of the rows in the original data that are not in the training dataset.

In [ ]:
test = data[~data.index.isin(train.index)].copy()
test.shape

Since we are still using `statsmodels` (like we did for linear regression), we can still use the same approach to fitting the logistic regression model. In particular, we must first define a `formula` that specifies the regression model we want to fit. The following code block shows how we can define a simple formula that specifes we are trying to predict the target as a function of the sum of the feature variables.

In [ ]:
formula = f'{target} ~ {" + ".join(features)}'
formula

The following code block fits the regression model and prints a summary of the fit. Although the summary looks very similar to what we got for linear regression, a major change is that instead of the coefficients being expressed in the same units as the target, they now represent a log-odds ratio. For those interested in the mathematics, please refer to https://en.wikipedia.org/wiki/Logistic_regression. The key thing to know is that the values represent the effect of the variable of the likelihood of seeing a positive (1) value for the target variable. As the coefficient go up, a one unit increase in the feature is more likely to result in a positive class value. On the other hand, as the coefficient goes down, a one unit increase in the feature is more likely to result in a negative class value

In [ ]:
reg = smf.logit(formula, data=data).fit()
reg.summary()

The following code block generates predictions for *the probability* of each instance belonging to the positive class for both the training and testing datasets. What we hope to see is that higher values of this probability are more likely to result in postive values for the class.

In [ ]:
train['positive_probability'] = reg.predict(train)
test['positive_probability'] = reg.predict(test)

The following code block generates a 95% confidence interval plot for the mean value of the `positive probability` column as the target varies.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

sns.pointplot(
    data=train,
    x=target,
    y='positive_probability',
    ax=ax,
    capsize=0.1,
)
ax.spines[['right', 'top']].set_visible(False)
plt.show()

The key thing to note is that this model is not giving a prediction, but a probability. Thus, we can set a threshold to convert the probabilities into predictions. Typically, as you might guess, this threshold is set at `0.50`. The following code block uses this threshold to convert the probabilities to predictions and then uses the predictions to compute the accuracy of the model on the training and testing datasets.

In [ ]:
threshold = 0.50

train_predictions = (train['positive_probability'] >= threshold).astype(int)
train_accuracy = (train_predictions == train[target]).astype(int).mean()
print(f'{train_accuracy = :.2%}')

test_predictions = (test['positive_probability'] >= threshold).astype(int)
test_accuracy = (test_predictions == test[target]).astype(int).mean()
print(f'{test_accuracy = :.2%}')

In the previous cell, we use accuracy as the metric. Accuracy simply measure the proportion (or percent) of correct predictions. However, let's define more formally. To do so, let:
- **TP** denote **True Positives**: cases where the target value is 1 and the prediction is also 1
- **TN** denote **True Negatives**: cases where the target value is 0 and the prediction is also 0
- **FP** denote **False Positives**: cases where the target value is 0 but the prediction is 1
- **FN** denote **False Negatives**: cases where the target value is 1 but the prediction is 0

Using this notation, we can define **Accuracy** as:
$$\displaystyle \frac{TP + TN}{TP + TN + FP + FN}.$$

The following code block runs a simple experiment that looks at the accuracy that we achieve on the two datasets as we vary the threshold used for converting the probabilities to a prediction. A vertical line is drawn at 0.50.

In [ ]:
threshold_experiment = {}

for threshold in np.arange(0.01, 1.0, 0.01):

    train_predictions = (train['positive_probability'] >= threshold).astype(int)
    train_accuracy = (train_predictions == train[target]).astype(int).mean()

    test_predictions = (test['positive_probability'] >= threshold).astype(int)
    test_accuracy = (test_predictions == test[target]).astype(int).mean()
    
    threshold_experiment[round(float(threshold), 2)] = {
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy
    }
    
threshold_experiment = pd.DataFrame().from_dict(
    threshold_experiment,
    orient='index',
)

threshold_experiment = threshold_experiment.reset_index()
threshold_experiment = threshold_experiment.rename(columns={'index': 'threshold'})

fig, ax = plt.subplots(1, 1, figsize=(10, 4))

ax.plot(
    threshold_experiment['threshold'],
    threshold_experiment['train_accuracy'],
    label='Train',
)

ax.plot(
    threshold_experiment['threshold'],
    threshold_experiment['test_accuracy'],
    label='Test',
)
ax.axvline(0.5)
ax.set_xlabel('threshold')
ax.set_ylabel('accuracy')
ax.legend()
ax.spines[['right', 'top']].set_visible(False)

plt.show()

From an accuracy perspective, the previous experiment shows that 0.5 is a reasonable choice. **However, is it the best choice for this problem?**

Whenever you are considering a classification problem, you must consider whether the costs of errors are the same. In this case, a **False Positive** corresponds to a case where we tell a patient they are likely to have diabetes when they don't. On the other hand, a **False Negative** corresponds to a case where we tell a patient they have nothing to worry about when in fact they have a serious disease. 

Let's see how the threshold of 0.5 does with respect to these error types.

In [ ]:
threshold = 0.50

test_predictions = (test['positive_probability'] >= threshold).astype(int)
test_TP = ((test_predictions == 1) & (test[target] == 1)).astype(int).sum()
print(f'{test_TP = }')

test_TN = ((test_predictions == 0) & (test[target] == 0)).astype(int).sum()
print(f'{test_TN = }')

test_FP = ((test_predictions == 1) & (test[target] == 0)).astype(int).sum()
print(f'{test_FP = }')

test_FN = ((test_predictions == 0) & (test[target] == 1)).astype(int).sum()
print(f'{test_FN = }')

As you can see, the current threshold favors false negatives. Let's rerun the experiment looking at how chaning the threshold affects the false positive and false negative rates on the testing dataset. Again, the 0.5 threshold is plotted for reference.

In [ ]:
threshold_experiment = {}

for threshold in np.arange(0.01, 1.0, 0.01):

    test_predictions = (test['positive_probability'] >= threshold).astype(int)
    test_FP = ((test_predictions == 1) & (test[target] == 0)).astype(int).sum()
    test_FN = ((test_predictions == 0) & (test[target] == 1)).astype(int).sum()
    
    threshold_experiment[round(float(threshold), 2)] = {
        'FP': test_FP,
        'FN': test_FN
    }
    
threshold_experiment = pd.DataFrame().from_dict(
    threshold_experiment,
    orient='index',
)

threshold_experiment = threshold_experiment.reset_index()
threshold_experiment = threshold_experiment.rename(columns={'index': 'threshold'})

fig, ax = plt.subplots(1, 1, figsize=(10, 4))

ax.plot(
    threshold_experiment['threshold'],
    threshold_experiment['FP'],
    label='FP',
)

ax.plot(
    threshold_experiment['threshold'],
    threshold_experiment['FN'],
    label='FN',
)
ax.axvline(0.5)
ax.set_xlabel('threshold')
ax.set_ylabel('Count')
ax.legend()
ax.spines[['right', 'top']].set_visible(False)

plt.show()

The previous code block shows that the threshold choice that maximizes accuracy, may not be best if we want to minimize false negatives. This is a very important thing to note as the choice of the best model or best parameters for a model is not necessarily based on the model that if the most accurate, but instead on the model that results in minimal harm.